# Apéndice E. Uso de la librería HLSLib

La implementación de algoritmos de procesamiento digital de señales en hardware (para FPGA o ASIC) se puede realizar con declaraciones explícitas de unidades MAC (Multiplicación-Acumulación) y su interconexión usando lenguajes de descripción de hardware (HDL: _Hardware description language_) como Verilog o VHDL, o bien empleando una estrategia de síntesis de alto nivel (**HLS: _High-level Synthesis_**) a partir de algoritmos descritos en lenguaje C que se explicará en este apéndice.

A lo largo del libro se han presentado fragmentos de código en lenguaje C para la implementación de las diferentes técnicas. Estos mismos algoritmos se pueden adaptar para sintetizar una solución en hardware usando compiladores HLS que se encuentran disponibles en herramientas comerciales de síntesis de hardware.

Las representaciones de estos algoritmos en lenguaje C sintetizable se conocen con el nombre de representación en **lenguaje C algorítmico** y para usarlo basta con declarar las variables con tipos de datos especializados que sustituyen los tipos de datos ``int`` o ``float``. Usando librerías en C, que 
son proporcionadas por los fabricantes de las herramientas HLS, es posible usar cualquier compilador para generar una versión ejecutable que permita simular y verificar el funcionamiento del algoritmo. Posteriormente, cuando la verificación es correcta, se usa el compilador HLS para traducir el algoritmo descrito en lenguaje C algorítmico en un archivo RTL (_Register Transfer Logic_) en Verilog o VHDL, que permite sintesizar la solución en hardware para una FPGA o ASIC.

Para ilustrar como usar estas herramientas de HLS, emplearemos en este apéndice la biblioteca de código abierto HLSLib de Siemens que se puede descargar de manera libre de https://hlslibs.org/

Esta biblioteca tiene diferentes tipos de datos y construcciones básicas para algoritmos de procesamiento digital de señales y _machine learning_. En particular para este apéndice usaremos únicamente los tipos de datos básicos "**AC Datatypes**".

A continuación se ilustra como usar HSL-Lib para implementar un filtro FIR en C algorítmico usando punto fijo. 

En el Capítulo 6 se explicó lo que era la representación en punto y cómo reescribir un algoritmo en lenguaje C para correrlo en software. A continuación se mostrará cómo usar el punto fijo y librería HSL-Lib para crear una filtro FIR sintetizable en hardware. 

Si usa la biblioteca AC Datatypes de HSL-Lib para punto fijo, basta con incluir la siguiente librería en su programa en C:

```C
#include <ac_fixed.h>
```

y cambiar todos las declaraciones de variables por el tipo de dato

```C
ac_fixed<W, I, S, Q, O>
```

donde 

* ``W`` indica el número total de bits de la variable
* ``I`` es el número de bits de la parte entera
* ``S`` toma los valores ``true`` o ``false`` indicando si la variable es una representación con signo o sin signo, respectivamente
* ``Q`` indica la forma de cómo se cuantizarán los datos cuando a la variable se le asigna un valor flotante, lo cual puede hacerse truncando los bits ``AC_TRN`` o con redondeo ``AC_RND``
* ``O`` indica como se manejará el desborde (_overflow_) durante las operaciones, que puede ser ``AC_WRAP`` donde se descartan los bits superiores o ``AC_SAT`` donde el resultado se satura a la representación máxima que soporte la variable, o ``AC_SAT_ZERO`` donde en caso de presentarse saturación el resultado es cero.

Para escribir el programa en C algorítmico partimos del filtro de referencia en punto flotante:

![FIR-SIMD](../img/ape_fir_basecode.png)
 
La señal de entrada, ``x[n]``, se asume en el rango de excursión entre -1 y 1, por lo tanto, si se va a almacenar en 16bits, el tipo de dato debe ser $Q(1,0,15)$.  Por esta razón, en la línea 2 se cambia la definición por un tipo de dato que satisfaga esta especificación:

```C
ac_fixed<16,0,true,AC_RND,AC_SAT>
```

De esta forma, ``x`` será un vector con la representación $Q(1,0,15)$ donde se incluye control de _overflow_ por saturación y redondeo en caso de que se le asignen valores flotantes.

Como se explicó en el Capítulo 6, para los coeficientes se debe estimar el valor máximo que alcanzaría la salida en el peor de los casos, es decir, estimar ``y`` cuando la señal de entrada toma los valores 1, lo cual se puede estimar a través de:

$$ max = \sum_{k=0}^{N_{h}-1}|h[k]|$$

Si $max<1$, los coeficientes se pueden asociar a un número de punto fijo $Q(1,0,15)$, en caso contrario, hay que determinar el número de bits de la parte entera para evitar desborde. Por ejemplo, si $max=2.71$ significa que son necesarios dos bits para la parte entera, así que si los coeficientes se almacenan en punto fijo de 16bits, ``h[n]`` debe declararse como $Q(1,2,13)$. Una forma matemática de calcular el número de bits de la parte entera es usar el logaritmo en base dos del redondeo por encima de $max$.

Ahora, una vez determinado el tipo de dato de punto fijo para ``x`` y ``h``, es necesario declarar el tipo de dato para ``y`` (línea 7).  Es claro que ``y`` debe ser de 32 bits como mínimo dado a que ``x`` y ``h`` son de 16bits y el producto en la línea 10 genera un dato de 32bits. Aquí se aplican las reglas de multiplicación en punto fijo, es decir, si el número de bits de la parte fraccional de ``y`` será la suma del número de bits de la parte fraccional de ``x`` y ``h``. 

Por ejemplo, si ``x`` y ``h`` se declararon como:

```C
ac_fixed<16,2,true,AC_RND,AC_SAT> hn[Q] = { ... };
ac_fixed<16,0,true,AC_RND,AC_SAT> x[Q];
```

``h`` tiene 13 bits en la parte fraccionaria, ``x``, 15 bits, entonces se declara ``y`` como:

```C
ac_fixed<32,3,true,AC_RND,AC_SAT> y = 0;
```

Esta es la configuración inicial de ``y``. Realmente se puede jugar con diferentes combinaciones, por ejemplo, incluir más bits de guarda, más bits en la parte fraccionaria, todo esto es posible explorarlo usando C algorítmico.`

Para finalizar la implementación, es necesario que la función retorne un valor ``y`` en el rango que espera la solución final en punto fijo. Si el dato de entrada es de 16 bits en formato $Q(1,0,15)$ una primera aproximación es asumir que el dato de salida sea en el mismo formato. Esto quizás no sea cierto en algunas aplicaciones que amplifican la señal de salida. Si hay amplificación, debe explorar usando más bits en la parte entera para la definición de la señal de salida. Esto se hace cambiando la línea 15 por:

```C
ac_fixed<16,0,true,AC_RND,AC_SAT> yt = y;
```

Ahora, si la función está embebida en un algoritmo que soporta como entrada y salida datos en flotante 

```C
double filtrar(double xn)
```

y no puede ser alterada, es necesario que el valor almacenado en yt se convierta a flotante, para ello la modificación a la línea 15 serán las siguientes líneas:

```C
ac_fixed<16,0,true,AC_RND,AC_SAT> yt = y;
return yt.to_double();
```

La re-escritura del filtro original en C algoritmo usando HLS-Lib lucirá entonces como:

```C
#include "ac_fixed.h"

#define Q 401

//Coeficientes
const ac_fixed<16,1,true> hn[] = {
	 -0.000000, 
	 -0.000013,
	 ...
};

//Elementos de retardo de la entrada
ac_fixed<16,0,true,AC_RND,AC_SAT> x[Q];

double filtrar(double xn)
{	
	int k;
	ac_fixed<32,2,true,AC_RND,AC_SAT> y = 0;
	x[0] = xn;
	for(k=0;k<Q;k++) {
		y += x[k]*hn[k];
	}
	for(k=Q-1;k>0;k--) {
		x[k] = x[k-1];
	}
    ac_fixed<16,2,true,AC_RND,AC_SAT> yt = y;
    return yt.to_double();
}
```

Nótese que en esta implementación no fue necesario alterar las líneas de código que hacían las operaciones en punto flotante, solamente es necesario cambiar la declaración de las variables. Asimismo, al vector de coeficientes se le asignan directamente los valores en punto flotante y no es necesario hacerle un cuantización de manera manual. Esto simplifica muchísimo la prueba y depuración de algoritmos en punto fijo y permite obtener soluciones finales que pueden ser finalmente sintetizadas en hardware.

Este algoritmo puede ser probado en cualquier compilador de C, usando las librerías descargadas de https://hlslibs.org/. Sin embargo, estas librerías no deberían ser usadas para hacer implementaciones en software en tiempo real pues aumentan el tiempo de ejecución.

Para la síntesis en hardware de estas definiciones en C algorítmico requiere contar con licencias de un compilador HLS.